# Crypto bot metrics evaluation:

## Test data extraction from Binance

In [38]:
from src.get_historical_data import get_historical_data

LOOKBACK = 365 * 3
COIN = 'BTCUSDT'

df = get_historical_data(coin=COIN,lookback=LOOKBACK)
df

,date,BTCUSDT
0,2022-08-06,22954.21
1,2022-08-07,23174.39
2,2022-08-08,23810.00
3,2022-08-09,23149.95
4,2022-08-10,23954.05
...,...,...
1090,2025-07-31,115764.08
1091,2025-08-01,113297.93
1092,2025-08-02,112546.35
1093,2025-08-03,114208.80


### Plotting data results:

In [39]:
import plotly.express as px
px.line(df, x='date', y=COIN)

## Extracting exchange interest from YFinance

In [40]:
from src.get_treasury_rate import get_treasury_rate

df2 = get_treasury_rate(lookback=LOOKBACK)
df2

[*********************100%***********************]  1 of 1 completed
ERROR	Task(Task-2) yfinance:multi.py:download()- 
1 Failed download:
ERROR	Task(Task-2) yfinance:multi.py:download()- ['^IRX']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


,date,treasury_rate


## Testing metrics

In [41]:
from src.crypto_metrics import CryptoMetrics
metrics = CryptoMetrics(lookback=LOOKBACK)
rsi = metrics.calculate_rsi(df=df, coin=COIN)

### RSI:

In [42]:
fig = px.line(rsi, x='date', y=COIN, title=f'{COIN} RSI', labels={f'{COIN}': f'{COIN} Price', 'date': 'Date'})

# Add green dots for rsi_buy_COIN
fig.add_scatter(x=rsi[rsi[f'rsi_buy_{COIN}'] == 1]['date'], 
                y=rsi[rsi[f'rsi_buy_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='RSI Buy')

# Add red dots for rsi_sell_COIN
fig.add_scatter(x=rsi[rsi[f'rsi_sell_{COIN}'] == 1]['date'], 
                y=rsi[rsi[f'rsi_sell_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='RSI Sell')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Treasury Rate correlation:

In [43]:
from importlib import reload
import src.crypto_metrics
reload(src.crypto_metrics)
from src.crypto_metrics import CryptoMetrics
treasury_corr = metrics.calculate_corr_treasury(df=df, coin=COIN)
treasury_corr

[*********************100%***********************]  1 of 1 completed
ERROR	Task(Task-2) yfinance:multi.py:download()- 
1 Failed download:
ERROR	Task(Task-2) yfinance:multi.py:download()- ['^IRX']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


,date,BTCUSDT,treasury_rate,treasury_corr_BTCUSDT,treasury_corr_buy_BTCUSDT,treasury_corr_sell_BTCUSDT
0,2022-08-06,22954.21,NaN,NaN,False,False
1,2022-08-07,23174.39,NaN,NaN,False,False
2,2022-08-08,23810.00,NaN,NaN,False,False
3,2022-08-09,23149.95,NaN,NaN,False,False
4,2022-08-10,23954.05,NaN,NaN,False,False
...,...,...,...,...,...,...
1090,2025-07-31,115764.08,NaN,NaN,False,False
1091,2025-08-01,113297.93,NaN,NaN,False,False
1092,2025-08-02,112546.35,NaN,NaN,False,False
1093,2025-08-03,114208.80,NaN,NaN,False,False


In [44]:
px.line(treasury_corr, x='date', y=f'treasury_corr_{COIN}')

In [45]:
fig = px.line(
    treasury_corr, 
    x='date', 
    y=COIN, 
    title=f'{COIN} correlation with interest rate USA', 
    labels= {
        COIN: f'{COIN} Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_COIN
fig.add_scatter(x=treasury_corr[treasury_corr[f'treasury_corr_buy_{COIN}'] == 1]['date'], 
                y=treasury_corr[treasury_corr[f'treasury_corr_buy_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='Treasury rate correlation buy signal')

# Add red dots for rsi_sell_COIN
fig.add_scatter(x=treasury_corr[treasury_corr[f'treasury_corr_sell_{COIN}'] == 1]['date'], 
                y=treasury_corr[treasury_corr[f'treasury_corr_sell_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='Treasury rate correlation sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Bollinger Bands:

In [46]:
bb = metrics.calculate_bollinger_bands(df=df, coin=COIN)
bb

,date,BTCUSDT,std_BTCUSDT,mean_BTCUSDT,top_band_BTCUSDT,bottom_band_BTCUSDT,bb_buy_BTCUSDT,bb_sell_BTCUSDT
0,2022-08-06,22954.21,NaN,22954.210000,NaN,NaN,0,0
1,2022-08-07,23174.39,155.690771,23064.300000,23297.836157,22830.763843,0,0
2,2022-08-08,23810.00,444.382686,23312.866667,23979.440695,22646.292638,0,0
3,2022-08-09,23149.95,371.868401,23272.137500,23829.940102,22714.334898,0,0
4,2022-08-10,23954.05,443.526225,23408.520000,24073.809338,22743.230662,0,0
...,...,...,...,...,...,...,...,...
1090,2025-07-31,115764.08,997.943782,118080.439333,119577.355007,116583.523660,1,0
1091,2025-08-01,113297.93,1542.413159,117688.464000,120002.083738,115374.844262,1,0
1092,2025-08-02,112546.35,2031.246206,117329.898000,120376.767308,114283.028692,1,0
1093,2025-08-03,114208.80,2177.243733,117087.818000,120353.683600,113821.952400,0,0


In [47]:
fig = px.line(
    bb, 
    x='date', 
    y=COIN, 
    title=f'Bollinger Bands for {COIN}', 
    labels= {
        COIN: f'{COIN} Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_COIN
fig.add_scatter(x=bb[bb[f'bb_buy_{COIN}'] == 1]['date'], 
                y=bb[bb[f'bb_buy_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='Bollinger Bands buy signal')

# Add red dots for rsi_sell_COIN
fig.add_scatter(x=bb[bb[f'bb_sell_{COIN}'] == 1]['date'], 
                y=bb[bb[f'bb_sell_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='Bollinger Bands sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Mean Average Convergence Divergence (MACD):

In [48]:
macd = metrics.calculate_macd(df=df, coin=COIN)
macd

,date,BTCUSDT,EMA12_BTCUSDT,EMA26_BTCUSDT,MACD_BTCUSDT,Signal_Line_BTCUSDT,MACD_buy_BTCUSDT,MACD_sell_BTCUSDT
0,2022-08-06,22954.21,22954.210000,22954.210000,0.000000,0.000000,0,0
1,2022-08-07,23174.39,22988.083846,22970.519630,17.564217,3.512843,1,0
2,2022-08-08,23810.00,23114.532485,23032.703361,81.829124,19.176100,0,0
3,2022-08-09,23149.95,23119.981334,23041.388297,78.593037,31.059487,0,0
4,2022-08-10,23954.05,23248.299590,23108.992868,139.306722,52.708934,0,0
...,...,...,...,...,...,...,...,...
1090,2025-07-31,115764.08,117583.509518,116031.157600,1552.351918,2175.201020,0,0
1091,2025-08-01,113297.93,116924.189592,115828.696296,1095.493296,1959.259475,0,0
1092,2025-08-02,112546.35,116250.675808,115585.559534,665.116275,1700.430835,0,0
1093,2025-08-03,114208.80,115936.541069,115483.577346,452.963723,1450.937413,0,0


In [49]:
fig = px.line(
    macd, 
    x='date', 
    y=COIN, 
    title=f'MACD for {COIN}', 
    labels= {
        COIN: f'{COIN} Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_COIN
fig.add_scatter(x=macd[macd[f'MACD_buy_{COIN}'] == 1]['date'], 
                y=macd[macd[f'MACD_buy_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='MACD buy signal')

# Add red dots for rsi_sell_COIN
fig.add_scatter(x=macd[macd[f'MACD_sell_{COIN}'] == 1]['date'], 
                y=macd[macd[f'MACD_sell_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='MACD sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Final buy setup:

In [50]:
df_buy = metrics.set_buy(df, coin=COIN)
df_buy

[*********************100%***********************]  1 of 1 completed
ERROR	Task(Task-2) yfinance:multi.py:download()- 
1 Failed download:
ERROR	Task(Task-2) yfinance:multi.py:download()- ['^IRX']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


,date,BTCUSDT,buy_BTCUSDT,sell_BTCUSDT
0,2022-08-06,22954.21,0,0
1,2022-08-07,23174.39,0,0
2,2022-08-08,23810.00,0,0
3,2022-08-09,23149.95,0,0
4,2022-08-10,23954.05,0,0
...,...,...,...,...
1090,2025-07-31,115764.08,0,0
1091,2025-08-01,113297.93,0,0
1092,2025-08-02,112546.35,0,0
1093,2025-08-03,114208.80,0,0


In [ ]:
fig = px.line(
    df_buy, 
    x='date', 
    y=COIN, 
    title=f'Buy setup for {COIN}', 
    labels= {
        COIN: f'{COIN} Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_COIN
fig.add_scatter(x=df_buy[df_buy[f'buy_{COIN}'] == 1]['date'], 
                y=df_buy[df_buy[f'buy_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='Buy signal')

# Add red dots for rsi_sell_COIN
fig.add_scatter(x=df_buy[df_buy[f'sell_{COIN}'] == 1]['date'], 
                y=df_buy[df_buy[f'sell_{COIN}'] == 1][COIN], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='Sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

## Evaluate models returns:

To evaluate the model return, it is possible to use a simulation of a bot working on the operations of trading for a given historic period of the coin price. Based on that, every trade will be set to operate with 10 dolars. By the end of the period, it will be computed the return of that.

Every buy will be 10 dollars
Every sell will be 10 dollars

The start value for investment will be 1000 dollars.

In [52]:
from src.simulations import simulate_model_trader

final_trader = simulate_model_trader(df=df_buy, initial_capital=1000, trade_value=10, coin=COIN)
print(f"Final balance: {final_trader['final_balance']:.2f}")

Initial Capital: $1000.00
Coin Holdings: 0.00000000 BTC
Coin Holdings in dollars: $0.00
Total Invested: $390.00
Final Balance: $1087.65
Total Profit/Loss: $87.65
Final balance: 1087.65


### Change of strategy:

Instead of selling the BTC, try just buying when the model advises to do:

In [53]:
from src.simulations import simulate_model_buyer

final_buyer = simulate_model_buyer(df_buy, initial_capital=1000, trade_value=10, coin=COIN)
print(f"Final balance: ${final_buyer['final_balance']:.2f}")

Initial Capital: $1000.00
Coin Holdings: 0.01116615 BTC
Coin Holdings in dollars: $1277.37
Total Invested: $390.00
Final Balance: $1887.37
Total Profit/Loss: $887.37
Final balance: $1887.37


### Comparison with DCA:

In [54]:
from src.simulations import simulate_dca

final_dca = simulate_dca(df_buy, initial_capital=1000, trade_value=10, coin=COIN, investment_interval='30D')
print(f"Final balance: ${final_dca['final_balance']:.2f}")

Initial Capital: $1000.00
Coin Holdings: 0.00977974 BTC
Coin Holdings in dolars: $1118.77
Total Invested: $370.00
Final Balance: $1748.77
Total Profit/Loss: $748.77
Final balance: $1748.77


### Calculating ROI of different strategies:

$$ ROI = (\frac{\text{Final Balance} - \text{Initial Capital}}{\text{Initial Capital}})\times 100$$

In [55]:
from src.simulations import roi

roi_trader = roi(invested_amount= final_trader['total_invested'], final_balance= final_trader['coin_holdings_dollars'])
roi_buyer = roi(invested_amount = final_buyer['total_invested'], final_balance= final_buyer['coin_holdings_dollars'])
roi_dca = roi(invested_amount = final_dca['total_invested'], final_balance= final_dca['coin_holdings_dollars'])

print(f"Trader ROI: {roi_trader:.2f}%")
print(f"Buyer ROI: {roi_buyer:.2f}%")
print(f"DCA ROI: {roi_dca:.2f}%")

Trader ROI: -100.00%
Buyer ROI: 227.53%
DCA ROI: 202.37%
